<a href="https://colab.research.google.com/github/hrishikeshmane/Skin_cancer_Classification/blob/master/Skin_cancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install kaggle


In [0]:
!mkdir .kaggle

In [0]:
from google.colab import files

In [0]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ihrishimane26","key":"d81b9e042ff925010f9909aa90b8311a"}'}

In [0]:
!mkdir -p ~/.kaggle

In [0]:
!cp kaggle.json ~/.kaggle/

In [0]:
!kaggle datasets list

ref                                                     title                                              size  lastUpdated          downloadCount  
------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
chicago/chicago-copa-cases                              Chicago COPA Cases                                  3MB  2019-04-05 00:00:32           1321  
himanshupoddar/zomato-bangalore-restaurants             Zomato Bangalore Restaurants                       88MB  2019-03-31 09:48:43           1630  
sfinspiredu/synchrotron-data-set                        Synchrotron Data Set                               10KB  2019-05-27 19:54:29            265  
crisparada/brazilian-cities                             Brazilian Cities                                  935KB  2019-05-24 20:49:21            847  
taniaj/australian-election-2019-tweets                  Australian Election 2019 Tweets             

In [0]:
!kaggle datasets list -s Skin

ref                                                    title                                           size  lastUpdated          downloadCount  
-----------------------------------------------------  ---------------------------------------------  -----  -------------------  -------------  
kmader/skin-cancer-mnist-ham10000                      Skin Cancer MNIST: HAM10000                      3GB  2018-09-20 20:36:13          10204  
alxmamaev/minecraft-skins                              Minecraft skins                                 16MB  2018-05-28 18:32:56             96  
tsaideepak/skin-cancer                                 skin cancer                                    230MB  2019-03-23 19:39:19             82  
stephaniekerr/cit-biochemistry-lab-study-topical-acid  Chemical Exfoliators in Skincare                31KB  2018-03-01 18:42:40            208  
drscarlat/melanoma                                     melanoma                                         3GB  2018-12-02 04:0

In [0]:
!kaggle datasets download "fanconic/skin-cancer-malignant-vs-benign"

 98% 160M/162M [00:04<00:00, 26.7MB/s]
100% 162M/162M [00:04<00:00, 38.1MB/s]


In [0]:
ls 

kaggle.json  sample_data/  skin-cancer-malignant-vs-benign.zip


In [0]:
!unzip skin-cancer-malignant-vs-benign.zip

Archive:  skin-cancer-malignant-vs-benign.zip
   creating: data/
   creating: data/benign/
  inflating: data/benign/1202.jpg    
  inflating: data/benign/307.jpg     
  inflating: data/benign/1203.jpg    
  inflating: data/benign/327.jpg     
  inflating: data/benign/1204.jpg    
  inflating: data/benign/3.jpg       
  inflating: data/benign/1205.jpg    
  inflating: data/benign/310.jpg     
  inflating: data/benign/284.jpg     
  inflating: data/benign/287.jpg     
  inflating: data/benign/288.jpg     
  inflating: data/benign/290.jpg     
  inflating: data/benign/293.jpg     
  inflating: data/benign/295.jpg     
  inflating: data/benign/296.jpg     
  inflating: data/benign/297.jpg     
  inflating: data/benign/298.jpg     
  inflating: data/benign/299.jpg     
  inflating: data/benign/300.jpg     
  inflating: data/benign/301.jpg     
  inflating: data/benign/302.jpg     
  inflating: data/benign/304.jpg     
  inflating: data/benign/306.jpg     
  inflating: data/benign/308.jpg   

In [0]:
!pip install tf-nightly-gpu-2.0-preview

     |████████████████████████████████| 345.6MB 61kB/s 
     |████████████████████████████████| 440kB 41.9MB/s 
     |████████████████████████████████| 3.2MB 42.4MB/s 


In [0]:
import warnings
warnings.filterwarnings("ignore")

import os
import cv2
import glob
import json
import h5py
import shutil

import numpy as np 
import pandas as pd 
import seaborn as sns
from pathlib import Path
import matplotlib.pylab as pl
import matplotlib.pyplot as plt


from skimage.io import imread, imsave
from skimage.transform import resize
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.applications import resnet50, mobilenet, mobilenet_v2, vgg16, inception_v3
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Reshape
from tensorflow.keras.layers import Input, Flatten, Activation
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import top_k_categorical_accuracy
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras import backend as K
import tensorflow as tf
from tensorflow.keras import layers


color = sns.color_palette()
%matplotlib inline
%config InlineBackend.figure_format="svg"

In [0]:
os.environ['PYTHONHASHSEED'] = '0'
seed=1234

np.random.seed(seed)


In [0]:
train_data_path = Path("data/")

In [0]:
img_height, img_width, img_channels = 224,224,3
batch_size = 64
nb_classes = 7

In [0]:
data_gen = ImageDataGenerator(preprocessing_function=mobilenet_v2.preprocess_input)

# Make data generator for training and validation data
train_data_gen = data_gen.flow_from_directory(train_data_path, 
                                              target_size=(img_height, img_width), 
                                              class_mode="categorical", 
                                              shuffle=True, 
                                              batch_size=batch_size)

Found 3297 images belonging to 2 classes.


In [0]:
def get_fine_tuning_model(base_model, top_model, inputs, learning_type):
    if learning_type=='transfer_learning':
        print("Doing transfer learning")
        K.set_learning_phase(0)
        base_model.trainable = False
        features = base_model(inputs)
        outputs = top_model(features)
    else:
        print("Doing fine-tuning")
        features = base_model(inputs)
        outputs = top_model(features)
    return Model(inputs, outputs)

In [0]:
base_model = mobilenet_v2.MobileNetV2(input_shape=(img_height, img_width, img_channels),
                                 include_top=False, 
                                 weights='imagenet',
                                 pooling="avg")

9412608/9406464 [==============================] - 2s 0us/step


In [0]:


# Define a top model: extra layers that we are going to add on 
# top of our base network
feature_inputs = Input(shape=base_model.output_shape, name='top_model_input')
x = Reshape((1,1,1280), name='reshape_1')(feature_inputs)
x = Dropout(0.5,name='drop')(x)
x = Conv2D(2, (1,1), name="last_conv", 
           kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
x = Activation(activation="softmax", name="ouputs")(x)
outputs = Reshape((2,), name="reshape_2")(x)

# define the  model
top_model = Model(feature_inputs, outputs, name='top_model')


# get model for tranfser learning
inputs = Input(shape=(img_height, img_width, img_channels))
model = get_fine_tuning_model(base_model, top_model, inputs, "transfer_learning")
model.summary()



Doing transfer learning
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
mobilenetv2_1.00_224 (Model) (None, 1280)              2257984   
_________________________________________________________________
top_model (Model)            (None, 2)                 2562      
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________


In [0]:
def top_2(y_true, y_pred):
    return top_k_categorical_accuracy(y_true=y_true, y_pred=y_pred, k=2)

In [0]:


# compile the model and check it 
optimizer = Adam(0.001)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=['categorical_accuracy', top_2])



In [0]:
# always use earlystopping
# the restore_best_weights parameter load the weights of the best iteration once the training finishes
es = EarlyStopping(patience=5, restore_best_weights=True)

# checkpoint to save model
chkpt = ModelCheckpoint(filepath="gdrive/My Drive/skin_cancer_data/skin_cancer_final.h5", save_best_only=True, save_weights_only=True)

# reduce on plateau
rp = ReduceLROnPlateau(min_lr=1e-5, patience=2, factor=0.5, mode="max", monitor="val_categorical_accuracy")

# number of training and validation steps for training and validation
all_train_images = len(glob.glob("data/**/*.jpg"))


nb_train_steps =  all_train_images // batch_size


# number of epochs 
nb_epochs=100
print("Number of training steps: ", nb_train_steps)



Number of training steps:  51


In [0]:
# train the model 
history1 = model.fit_generator(train_data_gen, 
                              epochs=nb_epochs, 
                              steps_per_epoch=nb_train_steps, 
                              callbacks=[es,chkpt, rp])



Epoch 1/100


W0615 17:16:42.646347 140171210594176 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/math_grad.py:1251: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


50/51 [============================>.] - ETA: 0s - loss: 0.5249 - categorical_accuracy: 0.7447 - top_2: 1.0000

W0615 17:16:58.019869 140171210594176 callbacks.py:1254] Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,categorical_accuracy,top_2
W0615 17:16:58.021986 140171210594176 callbacks.py:991] Can save best model only with val_loss available, skipping.
W0615 17:16:58.030197 140171210594176 callbacks.py:1828] Reduce LR on plateau conditioned on metric `val_categorical_accuracy` which is not available. Available metrics are: loss,categorical_accuracy,top_2,lr


51/51 [==============================] - 16s 307ms/step - loss: 0.5206 - categorical_accuracy: 0.7473 - top_2: 1.0000
Epoch 2/100
50/51 [============================>.] - ETA: 0s - loss: 0.3909 - categorical_accuracy: 0.8220 - top_2: 1.0000

...

51/51 [==============================] - 11s 221ms/step - loss: 0.1659 - categorical_accuracy: 0.9545 - top_2: 1.0000
Epoch 99/100
50/51 [============================>.] - ETA: 0s - loss: 0.1647 - categorical_accuracy: 0.9565 - top_2: 1.0000

W0615 17:35:26.016197 140171210594176 callbacks.py:1254] Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,categorical_accuracy,top_2
W0615 17:35:26.021069 140171210594176 callbacks.py:991] Can save best model only with val_loss available, skipping.
W0615 17:35:26.025408 140171210594176 callbacks.py:1828] Reduce LR on plateau conditioned on metric `val_categorical_accuracy` which is not available. Available metrics are: loss,categorical_accuracy,top_2,lr


51/51 [==============================] - 11s 222ms/step - loss: 0.1638 - categorical_accuracy: 0.9567 - top_2: 1.0000
Epoch 100/100
50/51 [============================>.] - ETA: 0s - loss: 0.1639 - categorical_accuracy: 0.9587 - top_2: 1.0000

W0615 17:35:37.283790 140171210594176 callbacks.py:1254] Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,categorical_accuracy,top_2
W0615 17:35:37.285722 140171210594176 callbacks.py:991] Can save best model only with val_loss available, skipping.
W0615 17:35:37.291245 140171210594176 callbacks.py:1828] Reduce LR on plateau conditioned on metric `val_categorical_accuracy` which is not available. Available metrics are: loss,categorical_accuracy,top_2,lr


51/51 [==============================] - 11s 221ms/step - loss: 0.1636 - categorical_accuracy: 0.9586 - top_2: 1.0000


In [0]:
# prepare validation data 
valid_images, valid_labels = [], []
class_indices = {'akiec': 0, 'bcc': 1, 'bkl': 2, 'df': 3, 'mel': 4, 'nv': 5, 'vasc': 6}

subdirs = sorted(os.listdir(valid_data_path))
subdirs = [x for x in subdirs if "DS_Store" not in x]

for subdir in subdirs:
  print(f"Processing {subdir}")
  images = os.listdir(valid_data_path / subdir)
  images = [x for x in images if "DS_Store" not in x]
  print("Number of images found: ", len(images))
  
  for img in images:
    img = imread(valid_data_path / subdir / img)
    img = cv2.resize(img, (img_height, img_width))
    label = class_indices[subdir]
    valid_images.append(img)
    valid_labels.append(label)
  print("="*50)
  
valid_images = np.array(valid_images, dtype=np.float32)
valid_images = mobilenet_v2.preprocess_input(valid_images)
valid_labels = np.array(valid_labels)
valid_labels_cat = to_categorical(valid_labels, num_classes=7)
print("\n", valid_images.shape, valid_labels.shape, valid_labels_cat.shape)

AttributeError: ignored

In [0]:
# save the model for future use
model.save("/skin_cancer_best_model.h5")

In [0]:


# save the architecture as well for future use 
json_string = model.to_json()

f = open("/skin_cancer_best_model_arch.json", "w")
json.dump(json_string, f)
f.close()



In [0]:
keras_file = "keras_model.h5"
tf.keras.models.save_model(model, keras_file)


In [34]:
# Convert to TensorFlow Lite model.
converter = tf.lite.TFLiteConverter.from_keras_model_file("/skin_cancer_best_model.h5")
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)


AttributeError: ignored